### Efficient Frontier 생성

In [28]:
import numpy as np
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt
# import cvxpy as cp
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy.spatial import distance
%matplotlib inline

In [29]:
df = pd.read_csv('C:\\Users\\kndon\\Desktop\\Felab\\finance_data.csv',index_col=0, parse_dates=True, encoding='utf-8')
df.reset_index(inplace = True)

In [30]:
df['Year'] = df['Date'].dt.strftime('%Y')
df['Month'] = df['Date'].dt.strftime('%m')
df['Day'] = df['Date'].dt.strftime('%d')
df

,Date,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,...,Rtail,Meals,Banks,Insur,RlEst,Fin,Other,Year,Month,Day
0,1975-01-02,1.37,2.82,4.14,2.21,1.88,4.85,2.23,2.35,1.63,...,4.02,2.14,3.17,2.28,5.20,2.00,1.60,1975,01,02
1,1975-01-03,2.76,1.36,1.64,2.33,0.25,0.43,1.54,1.84,-0.27,...,3.19,1.15,0.79,0.74,2.39,0.88,0.83,1975,01,03
2,1975-01-06,-2.42,0.69,2.08,3.84,0.66,1.24,2.53,1.97,0.43,...,0.92,1.37,1.29,1.27,4.93,1.54,0.67,1975,01,06
3,1975-01-07,2.12,-0.27,-0.53,1.08,-0.27,0.58,0.69,1.47,0.56,...,0.00,-0.97,0.58,0.09,4.14,0.15,0.00,1975,01,07
4,1975-01-08,-0.37,-0.80,-3.68,-0.44,-0.06,-1.70,-0.26,0.48,-1.61,...,-0.67,-3.94,-0.11,-0.28,1.24,-0.59,-1.00,1975,01,08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11345,2019-12-24,0.33,0.12,-0.29,-0.13,-0.28,1.37,-0.02,-0.37,0.09,...,0.19,0.20,0.14,-0.07,0.35,0.10,0.01,2019,12,24
11346,2019-12-26,1.09,-0.11,0.56,0.29,0.04,-0.57,0.24,0.33,0.08,...,1.67,0.34,0.65,0.15,0.63,0.46,0.33,2019,12,26
11347,2019-12-27,-0.15,0.50,0.46,0.43,0.19,-0.62,-0.55,-0.72,0.55,...,-0.03,0.09,-0.18,0.08,-0.31,-0.13,-0.24,2019,12,27
11348,2019-12-30,-0.90,-0.34,-0.19,-0.51,-0.83,-1.89,-1.14,-0.13,-0.84,...,-0.61,-0.52,-0.45,-0.24,0.15,-0.59,-0.20,2019,12,30


In [31]:
df.set_index('Date')

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Rtail,Meals,Banks,Insur,RlEst,Fin,Other,Year,Month,Day
Date,,,,,,,,,,,,,,,,,,,,,
1975-01-02,1.37,2.82,4.14,2.21,1.88,4.85,2.23,2.35,1.63,4.57,...,4.02,2.14,3.17,2.28,5.20,2.00,1.60,1975,01,02
1975-01-03,2.76,1.36,1.64,2.33,0.25,0.43,1.54,1.84,-0.27,1.31,...,3.19,1.15,0.79,0.74,2.39,0.88,0.83,1975,01,03
1975-01-06,-2.42,0.69,2.08,3.84,0.66,1.24,2.53,1.97,0.43,2.53,...,0.92,1.37,1.29,1.27,4.93,1.54,0.67,1975,01,06
1975-01-07,2.12,-0.27,-0.53,1.08,-0.27,0.58,0.69,1.47,0.56,1.04,...,0.00,-0.97,0.58,0.09,4.14,0.15,0.00,1975,01,07
1975-01-08,-0.37,-0.80,-3.68,-0.44,-0.06,-1.70,-0.26,0.48,-1.61,0.19,...,-0.67,-3.94,-0.11,-0.28,1.24,-0.59,-1.00,1975,01,08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,0.33,0.12,-0.29,-0.13,-0.28,1.37,-0.02,-0.37,0.09,0.03,...,0.19,0.20,0.14,-0.07,0.35,0.10,0.01,2019,12,24
2019-12-26,1.09,-0.11,0.56,0.29,0.04,-0.57,0.24,0.33,0.08,0.50,...,1.67,0.34,0.65,0.15,0.63,0.46,0.33,2019,12,26
2019-12-27,-0.15,0.50,0.46,0.43,0.19,-0.62,-0.55,-0.72,0.55,0.30,...,-0.03,0.09,-0.18,0.08,-0.31,-0.13,-0.24,2019,12,27


In [32]:
df1 = df.groupby(['Year','Month']).mean()
df1

Agric     Food      Soda      Beer      Smoke     Toys   \
Year Month                                                               
1975 01     0.395000  0.740000  1.092727  1.058182  0.258636  1.117727   
     02     0.130526  0.222632  0.341053  0.554737 -0.168421  0.308947   
     03    -0.005500  0.136000  0.677500  0.247000  0.291000  0.652000   
     04     0.299091  0.020455  0.179545 -0.167273  0.227273  0.465455   
     05     0.110476  0.280952  0.434762  0.250000  0.099048  0.038571   
...              ...       ...       ...       ...       ...       ...   
2019 08    -0.023636  0.001818  0.125455  0.278182 -0.511818  0.116818   
     09    -0.201500  0.080500 -0.023000  0.087000  0.105500 -0.302000   
     10    -0.189565 -0.087391 -0.016957 -0.043478  0.343478  0.260870   
     11    -0.024500  0.168000 -0.008000 -0.028500  0.273000  0.214000   
     12     0.518571  0.145238  0.187619  0.067143  0.152381 -0.036667   

               Fun       Books     Hshld     Clths  ...     Boxes     Trans  \
Year Month                                          ...                       
1975 01     1.435909  0.959091  0.744091  1.294545  ...  0.285455  0.750909   
     02     1.058947  0.553158  0.585789  0.001053  ...  0.588947  0.292632   
     03     0.795500  0.494000  0.184500  1.035500  ... -0.137000  0.207000   
     04     0.286818  0.335455  0.303182  0.244091  ...  0.471818  0.204545   
     05     0.323810  0.181905  0.119048  0.104762  ...  0.354286 -0.017619   
...              ...       ...       ...       ...  ...       ...       ...   
2019 08    -0.331364 -0.197727  0.080000 -0.222273  ...  0.092273 -0.322727   
     09    -0.203500  0.038000  0.088500  0.438500  ... -0.043000  0.030000   
     10     0.251739  0.134348 -0.028261 -0.082609  ...  0.042174  0.067391   
     11     0.315000 -0.052500 -0.060500  0.226500  ...  0.092000  0.134500   
     12     0.234762  0.156190  0.124286  0.350476  ...  0.030000 -0.002857   

               Whlsl     Rtail     Meals     Banks     Insur     RlEst  \
Year Month                                                               
1975 01     0.739091  1.116364  1.157727  0.922727  0.448182  1.779545   
     02     0.330526  0.202105  0.779474 -0.100526  0.208421 -0.015263   
     03     0.375500  0.531500  0.633500  0.098500  0.157500  0.777500   
     04     0.219545  0.045000  0.629091  0.092273 -0.115909 -0.494091   
     05     0.158095  0.217143  0.190952  0.267619  0.505238  0.207619   
...              ...       ...       ...       ...       ...       ...   
2019 08    -0.096364  0.007727  0.070455 -0.223182 -0.213182 -0.146364   
     09     0.266000  0.048500 -0.120500  0.197500  0.006000  0.146000   
     10     0.110870  0.076087 -0.190000  0.163913  0.166087  0.133913   
     11     0.216500  0.102000  0.139000  0.249500  0.311000  0.383000   
     12     0.117619  0.059524  0.116667  0.154286  0.103810  0.223333   

               Fin       Other  
Year Month                      
1975 01     0.690455  0.670455  
     02     0.024211  0.215263  
     03     0.117500  0.137500  
     04     0.150455  0.202273  
     05     0.231905  0.262857  
...              ...       ...  
2019 08    -0.155909 -0.146818  
     09     0.124000  0.111500  
     10     0.042609  0.110435  
     11     0.344500  0.223500  
     12     0.035238  0.093810  

[540 rows x 49 columns]

In [33]:
prets = []
pvols = []
stocks_6m = []
stocks_1m = []
for i in range(540):
    stk = df1.iloc[i]
    stocks_1m.append(stk)
    
    stock = df1.iloc[i:i+6]
    stocks_6m.append(stock)

In [34]:
for _ in range(6):
    del stocks_1m[0]


for _ in range(6):
    del stocks_6m[-1]


In [35]:
stocks_1m

[Agric   -0.163182
 Food    -0.186818
 Soda    -0.449545
 Beer    -0.200909
 Smoke   -0.309091
 Toys    -0.284545
 Fun     -0.311818
 Books   -0.261364
 Hshld   -0.374091
 Clths   -0.162273
 Hlth    -0.409545
 MedEq   -0.426364
 Drugs   -0.617273
 Chems   -0.203636
 Rubbr   -0.209091
 Txtls   -0.160455
 BldMt   -0.210455
 Cnstr   -0.482727
 Steel   -0.306364
 FabPr   -0.123636
 Mach    -0.212273
 ElcEq   -0.295455
 Autos    0.061818
 Aero    -0.364545
 Ships   -0.022727
 Guns    -0.175000
 Gold    -0.338182
 Mines   -0.162273
 Coal    -0.138636
 Oil     -0.195000
 Util    -0.220909
 Telcm   -0.194091
 PerSv   -0.097727
 BusSv   -0.337727
 Hardw   -0.457727
 Softw    0.771818
 Chips   -0.399545
 LabEq   -0.459091
 Paper   -0.003636
 Boxes   -0.485909
 Trans   -0.118182
 Whlsl   -0.062727
 Rtail   -0.385455
 Meals   -0.416364
 Banks   -0.312727
 Insur   -0.409545
 RlEst   -0.425000
 Fin     -0.332273
 Other   -0.217273
 Name: (1975, 07), dtype: float64,
 Agric   -0.449524
 Food    -0.038

In [37]:
print(len(stocks_6m),len(stocks_1m))

534 534


In [38]:
stocks_6m[-1]

Agric     Food      Soda      Beer      Smoke     Toys   \
Year Month                                                               
2019 06     0.515500  0.238500  0.208500  0.259500  0.057000  0.460500   
     07     0.040000  0.113636  0.142273 -0.094545  0.164545  0.601364   
     08    -0.023636  0.001818  0.125455  0.278182 -0.511818  0.116818   
     09    -0.201500  0.080500 -0.023000  0.087000  0.105500 -0.302000   
     10    -0.189565 -0.087391 -0.016957 -0.043478  0.343478  0.260870   
     11    -0.024500  0.168000 -0.008000 -0.028500  0.273000  0.214000   

               Fun       Books     Hshld     Clths  ...     Boxes     Trans  \
Year Month                                          ...                       
2019 06     0.359500  0.424000  0.334000  0.441000  ...  0.488500  0.328500   
     07    -0.168182  0.152727  0.223636  0.082273  ...  0.069545  0.092273   
     08    -0.331364 -0.197727  0.080000 -0.222273  ...  0.092273 -0.322727   
     09    -0.203500  0.038000  0.088500  0.438500  ... -0.043000  0.030000   
     10     0.251739  0.134348 -0.028261 -0.082609  ...  0.042174  0.067391   
     11     0.315000 -0.052500 -0.060500  0.226500  ...  0.092000  0.134500   

               Whlsl     Rtail     Meals     Banks     Insur     RlEst  \
Year Month                                                               
2019 06     0.364000  0.350000  0.370500  0.342500  0.207000  0.552000   
     07     0.024091  0.024091  0.138636  0.164545  0.095455  0.158636   
     08    -0.096364  0.007727  0.070455 -0.223182 -0.213182 -0.146364   
     09     0.266000  0.048500 -0.120500  0.197500  0.006000  0.146000   
     10     0.110870  0.076087 -0.190000  0.163913  0.166087  0.133913   
     11     0.216500  0.102000  0.139000  0.249500  0.311000  0.383000   

               Fin       Other  
Year Month                      
2019 06     0.312500  0.356500  
     07     0.106818 -0.099091  
     08    -0.155909 -0.146818  
     09     0.124000  0.111500  
     10     0.042609  0.110435  
     11     0.344500  0.223500  

[6 rows x 49 columns]

In [39]:
stocks_1m[-1]

Agric    0.518571
Food     0.145238
Soda     0.187619
Beer     0.067143
Smoke    0.152381
Toys    -0.036667
Fun      0.234762
Books    0.156190
Hshld    0.124286
Clths    0.350476
Hlth     0.147619
MedEq    0.070000
Drugs    0.187619
Chems    0.074286
Rubbr    0.146190
Txtls   -0.155238
BldMt    0.116190
Cnstr   -0.120000
Steel    0.041429
FabPr    0.531429
Mach     0.183333
ElcEq    0.103333
Autos    0.350000
Aero    -0.253810
Ships    0.016667
Guns    -0.011429
Gold     0.668571
Mines    0.383333
Coal     0.050000
Oil      0.300000
Util     0.185238
Telcm    0.050952
PerSv    0.155238
BusSv    0.038095
Hardw    0.236190
Softw    0.092381
Chips    0.340952
LabEq    0.175714
Paper    0.131429
Boxes    0.030000
Trans   -0.002857
Whlsl    0.117619
Rtail    0.059524
Meals    0.116667
Banks    0.154286
Insur    0.103810
RlEst    0.223333
Fin      0.035238
Other    0.093810
Name: (2019, 12), dtype: float64

In [9]:
def portfolio_opt(data, r, verbose=True):
    mu = data.mean()
    cov = data.cov()
    n = len(data.columns)
    weight = cp.Variable(n)
    
    obj = cp.quad_form(weight, cov)
    
    prob = cp.Problem(cp.Minimize(obj), constraints = [cp.matmul(mu,weight) >= r, cp.sum(weight) == 1, weight >= 0])
    prob.solve(solver="CPLEX", verbose=verbose)
    return weight.value

In [10]:
def GMV(data, verbose=True):
    mu = data.mean()
    cov = data.cov()
    n = len(data.columns)
    weight = cp.Variable(n)
    
    obj = cp.quad_form(weight, cov)
    
    prob = cp.Problem(cp.Minimize(obj), constraints = [cp.sum(weight) == 1, weight >= 0])
    prob.solve(solver="CPLEX", verbose=verbose)
    return weight.value

In [13]:
mu_lists_est = []
std_lists_est = []
mu_lists_act = []
std_lists_act = []

In [14]:
result = pd.DataFrame()
for i in range(510):
    mu_est = stocks_6m[i].mean()
    cov_est = stocks_6m[i].cov()

    mu_act = stocks_1m[i].mean()
    cov_act = stocks_1m[i].cov()
    
    mu_list_est= []
    std_list_est= []
    mu_list_act= []
    std_list_act= []
    GMV_w = GMV(stocks_6m[i], verbose = False)
    GMV_mu = GMV_w@mu_est
    
    for tret in np.linspace(GMV_mu, max(mu_est), 30):
         
        weight = portfolio_opt(data=stocks_6m[i], r = tret, verbose = False)
        mu_list_est.append(weight@mu_est)
        std_list_est.append(np.sqrt(np.dot(weight, np.dot(cov_est, weight))))

        mu_list_act.append(weight@mu_act)
        std_list_act.append(np.sqrt(np.dot(weight, np.dot(cov_act, weight))))

   
    mu_lists_est.append(mu_list_est)
    std_lists_est.append(std_list_est)
    mu_lists_act.append(mu_list_act)
    std_lists_act.append(std_list_act)

#     print('------------------------------------------------')

현재 i는 0
현재 i는 1
현재 i는 2
현재 i는 3
현재 i는 4
현재 i는 5
현재 i는 6
현재 i는 7
현재 i는 8
현재 i는 9
현재 i는 10
현재 i는 11
현재 i는 12
현재 i는 13
현재 i는 14
현재 i는 15
현재 i는 16
현재 i는 17
현재 i는 18
현재 i는 19
현재 i는 20
현재 i는 21
현재 i는 22
현재 i는 23
현재 i는 24
현재 i는 25
현재 i는 26
현재 i는 27
현재 i는 28
현재 i는 29
현재 i는 30
현재 i는 31
현재 i는 32
현재 i는 33
현재 i는 34
현재 i는 35
현재 i는 36
현재 i는 37
현재 i는 38
현재 i는 39
현재 i는 40
현재 i는 41
현재 i는 42
현재 i는 43
현재 i는 44
현재 i는 45
현재 i는 46
현재 i는 47
현재 i는 48
현재 i는 49
현재 i는 50
현재 i는 51
현재 i는 52
현재 i는 53
현재 i는 54
현재 i는 55
현재 i는 56
현재 i는 57
현재 i는 58
현재 i는 59
현재 i는 60
현재 i는 61
현재 i는 62
현재 i는 63
현재 i는 64
현재 i는 65
현재 i는 66
현재 i는 67
현재 i는 68
현재 i는 69
현재 i는 70
현재 i는 71
현재 i는 72
현재 i는 73
현재 i는 74
현재 i는 75
현재 i는 76
현재 i는 77
현재 i는 78
현재 i는 79
현재 i는 80
현재 i는 81
현재 i는 82
현재 i는 83
현재 i는 84
현재 i는 85
현재 i는 86
현재 i는 87
현재 i는 88
현재 i는 89
현재 i는 90
현재 i는 91
현재 i는 92
현재 i는 93
현재 i는 94
현재 i는 95
현재 i는 96
현재 i는 97
현재 i는 98
현재 i는 99
현재 i는 100
현재 i는 101
현재 i는 102
현재 i는 103
현재 i는 104
현재 i는 105
현재 i는 106
현재 i는 107
현재 i는 108
현재 i는 109
현재 i는 110


In [15]:
df_est = {}
df_act = {}

df_est['evols'] = std_lists_est
df_est['erets'] = mu_lists_est

df_act['avols'] = std_lists_act
df_act['arets'] = mu_lists_act

df_estimate = pd.DataFrame.from_records(df_est)
df_estimate.to_csv('Estimate.csv')

df_actual = pd.DataFrame.from_records(df_act)
df_actual.to_csv('Actual.csv')

In [45]:
### scipy.spatial.distance.cdist의 거리를 구하는 함수들인데, 현재 배열형태에 맞게 수정이 필요함, 현재는 에러 발생
def E_D(evols,erets,avols,arets):
    dist = 0
    for i in range(len(evols)):
        dist += distance.euclidean([evols[i], erets[i]], [avols[i], arets[i]])
    return dist
#   AttributeError: 'float' object has no attribute 'euclidean'  에러 발생 ..

In [66]:
def C_D(evols,erets,avols,arets):
    dist = 0
    for i in range(len(evols)):
        a = distance.cosine([evols[i], erets[i]], [avols[i], arets[i]])
        dist += a
    return dist

In [47]:
def _c(ca, i, j, p, q):

    if ca[i, j] > -1:
        return ca[i, j]
    elif i == 0 and j == 0:
        ca[i, j] = np.linalg.norm(p[i]-q[j])
    elif i > 0 and j == 0:
        ca[i, j] = max(_c(ca, i-1, 0, p, q), np.linalg.norm(p[i]-q[j]))
    elif i == 0 and j > 0:
        ca[i, j] = max(_c(ca, 0, j-1, p, q), np.linalg.norm(p[i]-q[j]))
    elif i > 0 and j > 0:
        ca[i, j] = max(
            min(
                _c(ca, i-1, j, p, q),
                _c(ca, i-1, j-1, p, q),
                _c(ca, i, j-1, p, q)
            ),
            np.linalg.norm(p[i]-q[j])
            )
    else:
        ca[i, j] = float('inf')

    return ca[i, j]


def frdist(p, q):
    p = np.array(p, np.float64)
    q = np.array(q, np.float64)

    len_p = len(p)
    len_q = len(q)

    if len_p == 0 or len_q == 0:
        raise ValueError('Input curves are empty.')

    if len_p != len_q or len(p[0]) != len(q[0]):
        raise ValueError('Input curves do not have the same dimensions.')

    ca = (np.ones((len_p, len_q), dtype=np.float64) * -1)

    dist = _c(ca, len_p-1, len_q-1, p, q)
    return dist
#def form(evols,erets,avols,arets):
#    P = np.array([])
#    Q = np.array([])
#    for i in range(len(evols)):
#    P.append([evols,erets])
#    Q.append([avols,arets])
    
def distcal(evols,erets,avols,arets):
    distance = 0
    for i in range(len(evols)):
        print(frdist([evols[i],erets[i]],[avols[i],arets[i]]))
    return distance

def F_D(evols,erets,avols,arets):
    distance = 0
    P = np.empty((1,2), dtype=float)
    Q = np.empty((1,2), dtype=float)
    for i in range(len(evols)):
        P[0]=[evols[i],erets[i]]
        Q[0]=[avols[i],arets[i]]
        distance += frdist(P,Q)
    return distance

In [48]:
b = []
for j in range(len(mu_lists_est)):
    a = F_D(std_lists_est[j],mu_lists_est[j],std_lists_act[j],mu_lists_act[j])
    b.append(a)

[20.938391172476088, 9.911010675946617, 4.164048925705649, 8.043370823907617, 5.811686860713836, 8.379913016608432, 3.7429292320274903, 3.656728426597911, 6.747724009505415, 11.970934711572284, 2.7373529821790847, 6.829533468247801, 4.22726644833671, 3.394278263914928, 6.825306967979778, 4.2387077797756785, 4.3787370746851995, 4.4956746104453895, 6.433156185582829, 5.483937029451922, 3.148978211419519, 2.159754345057825, 3.951177902388636, 2.7731319105438876, 2.119893142365071, 2.1721097847970277, 4.654290064505475, 1.7364319817113045, 6.683636203212451, 4.1803887391149575, 2.9642378520310353, 3.3839985222974898, 2.4054744288483088, 2.9717626694628843, 5.389956256744977, 3.03398018820995, 9.796846592393262, 26.214165731207462, 5.613965105699491, 2.9700125627209384, 6.954429198548386, 4.4805884871420085, 4.2190144624546475, 3.449410098635035, 7.418543030069263, 2.796081324791923, 1.1015720792126216, 2.4723285340392818, 17.579112209472857, 4.020761898607398, 8.467649642743076, 5.10002596

In [49]:
Frechet = {}
Frechet['F_D'] = b

df_fd = pd.DataFrame.from_records(Frechet)
df_fd.to_csv('Frechet Distance.csv')

In [50]:
c = []
for k in range(len(mu_lists_est)):
    a = E_D(std_lists_est[k],mu_lists_est[k],std_lists_act[k],mu_lists_act[k])
    c.append(a)

In [51]:
Euclidean = {}
Euclidean['E_D'] = c

df_ed = pd.DataFrame.from_records(Euclidean)
df_ed.to_csv('Euclidean Distance.csv')

In [67]:
def C_D(evols,erets,avols,arets):
    dist = 0
    for i in range(len(evols)):
        a = distance.cosine([evols[i], erets[i]], [avols[i], arets[i]])
        dist += a
    return dist

In [68]:
d = []
for l in range(len(mu_lists_est)):
    a = C_D(std_lists_est[l], mu_lists_est[l], std_lists_act[l],mu_lists_act[l])
    d.append(a)

Cosige = {}
Cosige['C_D'] = c

df_ed = pd.DataFrame.from_records(Cosige)
df_ed.to_csv('Cosige Distance.csv')

In [54]:
mu_lists_est

[[0.2529856327077068,
  0.2764590018986032,
  0.2999469697993178,
  0.3234349377073551,
  0.34692290565278555,
  0.37041087357258706,
  0.39389884143502635,
  0.41738680934472805,
  0.4408747773283801,
  0.46436274520109677,
  0.4878507130674339,
  0.5113386810383879,
  0.5348266488887342,
  0.5583146168074755,
  0.5818025847021445,
  0.6052905526471384,
  0.6287785205185492,
  0.6522664884252664,
  0.6757544563346904,
  0.6992424242070129],
 [0.06971807730478281,
  0.08451770439560383,
  0.0993214346492344,
  0.11412516486601443,
  0.12892889504386743,
  0.14373262527067723,
  0.1585363555950551,
  0.17334008574534512,
  0.18814381596393956,
  0.20294754625612413,
  0.217751276421987,
  0.23255500666093967,
  0.24735873689424331,
  0.26216246711686686,
  0.2769661973388734,
  0.29176992756758485,
  0.30657365780984325,
  0.32137738802570665,
  0.3361811182571997,
  0.35098484844726296],
 [0.017929107017871523,
  0.037414255866357854,
  0.05690555142533325,
  0.07639684694099187,
  0.0

In [ ]:
def C_D(evols,erets,avols,arets):
    dist = 0
    for i in range(len(evols)):
        for j in range(len(evols[i]))
        dist += distance.cosine([evols[i][j], erets[i][j]], [avols[i][j], arets[i][j]])
    return dist

In [55]:
len(mu_lists_est)

510

In [58]:
dist = distance.cosine([0,1],[1,0])
print(dist)

1.0
